In [5]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import InceptionV3
from sklearn.metrics import classification_report

In [6]:
# Set the path to the main directory containing subdirectories with images
main_dir = r'C:\Users\Plaksha\3D Objects\DL\fma_img\fma_img'

# Define image dimensions and batch size
img_height = 100
img_width = 100
batch_size = 16

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use subset parameter for training data
)

validation_generator = train_datagen.flow_from_directory(
    main_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use subset parameter for validation data
)


Found 230 images belonging to 13 classes.
Found 50 images belonging to 13 classes.


In [8]:

# Define the model architecture
base_model = InceptionV3(include_top=False, input_shape=(img_height, img_width, 3))

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=20, validation_data=validation_generator)

# Print accuracy and other performance metrics
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])
print("Training Loss:", history.history['loss'][-1])
print("Validation Loss:", history.history['val_loss'][-1])

# Evaluate the model
validation_steps = len(validation_generator)
predictions = model.predict(validation_generator, steps=validation_steps)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
# Print classification report with specified labels parameter
# print(classification_report(true_classes, predicted_classes, labels=np.unique(true_classes), target_names=class_labels))

Epoch 1/20
15/15 [==============================] - 30s 885ms/step - loss: 2.6931 - accuracy: 0.1348 - val_loss: 13.3105 - val_accuracy: 0.1000
Epoch 2/20
15/15 [==============================] - 12s 779ms/step - loss: 2.1631 - accuracy: 0.2783 - val_loss: 7.4073 - val_accuracy: 0.1000
Epoch 3/20
15/15 [==============================] - 12s 824ms/step - loss: 1.9626 - accuracy: 0.3826 - val_loss: 9.9717 - val_accuracy: 0.1000
Epoch 4/20
15/15 [==============================] - 13s 862ms/step - loss: 1.6458 - accuracy: 0.5174 - val_loss: 2.3060 - val_accuracy: 0.1600
Epoch 5/20
15/15 [==============================] - 12s 808ms/step - loss: 1.6061 - accuracy: 0.5130 - val_loss: 2.2816 - val_accuracy: 0.1400
Epoch 6/20
15/15 [==============================] - 12s 808ms/step - loss: 1.4468 - accuracy: 0.5739 - val_loss: 2.7666 - val_accuracy: 0.3000
Epoch 7/20
15/15 [==============================] - 13s 837ms/step - loss: 1.1212 - accuracy: 0.7217 - val_loss: 2.2055 - val_accuracy: 0.300

In [ ]:
# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to calculate embeddings
def calculate_embeddings(image_path):
    img_array = preprocess_image(image_path)
    embeddings = model.predict(img_array)
    return embeddings

# Function to open file dialog and get the image path
def browse_file():
    file_path = filedialog.askopenfilename()
    image_path_var.set(file_path)
    if file_path:
        img = Image.open(file_path)
        img = img.resize((200, 200), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img

# Function to match embeddings and determine class
def match_embeddings():
    image_path = image_path_var.get()
    if image_path:
        uploaded_embeddings = calculate_embeddings(image_path)
        min_distance = float('inf')
        min_class = None
        for subdir in os.listdir(main_dir):
            subdir_path = os.path.join(main_dir, subdir)
            for img_file in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, img_file)
                embeddings = calculate_embeddings(img_path)
                distance = np.linalg.norm(uploaded_embeddings - embeddings)
                if distance < min_distance:
                    min_distance = distance
                    min_class = subdir
        result_var.set(f"The song clip belongs to class: {min_class}")

# Create tkinter window
root = tk.Tk()
root.title("Image Classification")

# Create widgets
browse_button = tk.Button(root, text="Upload Image", command=browse_file)
browse_button.pack(pady=10)

image_label = tk.Label(root)
image_label.pack(pady=10)

image_path_var = tk.StringVar()
image_path_entry = tk.Entry(root, textvariable=image_path_var, state='readonly')
image_path_entry.pack(pady=10)

classify_button = tk.Button(root, text="Classify Image", command=match_embeddings)
classify_button.pack(pady=10)

result_var = tk.StringVar()
result_label = tk.Label(root, textvariable=result_var)
result_label.pack(pady=10)

# Function to close the tkinter window
def close_window():
    root.destroy()

# Bind closing event to close_window function
root.protocol("WM_DELETE_WINDOW", close_window)

# Run the tkinter event loop
root.mainloop()